Wanted LaaS의 Vector Search 예제입니다.

LaaS 👉 https://laas.wanted.co.kr/

Swagger Documents 👉 https://api-laas.wanted.co.kr/api/vector-swagger-ui/swagger-ui/index.html

In [ ]:
import requests
import json

# 기본 상수
BASE_URL = 'https://api-laas.wanted.co.kr/api'
HEADERS = {'Content-Type': 'application/json'}
API_KEY = 'your_api_key_here'
PROJECT = 'your_project_name_here'

# 인증을 위한 API KEY와 프로젝트 코드 (LaaS 대시보드에서 확인)
HEADERS['apiKey'] = API_KEY
HEADERS['project'] = PROJECT

In [ ]:
# 컬렉션 생성
def create_collection(collection_name: str):
    url = f"{BASE_URL}/collection/{collection_name}"
    data = {}
    response = requests.post(url, headers=HEADERS, data=json.dumps(data))
    return response.json()

In [ ]:
# 새 문서 추가: doc_id는 컬렉션 내에서 유일해야합니다.
def add_document(collection_name: str, doc_id: int, text: str, metadata: dict):
    url = f"{BASE_URL}/document/{collection_name}"
    data = {
        "ids": [doc_id],
        "texts": [text],
        "metadata": [metadata]
    }
    response = requests.put(url, headers=HEADERS, data=json.dumps(data))
    return response.json()

In [ ]:
# ID로 유사 문서 검색: add_document에서 사용한 ID의 문서와 유사한 문서를 검색합니다.
def search_similar_documents_by_id(collection_name: str, doc_id: int):
    url = f"{BASE_URL}/document/{collection_name}/similar/{doc_id}"
    data = {
        "limit": 10,  # [필수] 검색할 문서 수
        "offset": 0,  # [필수] 페이징에 사용할 offset 값
        "with_metadata": True,  # [선택] 응답에 메타데이터 반환 여부
        "with_vector": False  # [선택] 응답에 내부 벡터 반환 여부
    }
    response = requests.post(url, headers=HEADERS, data=json.dumps(data))
    return response.json()

In [ ]:
# 입력 텍스트와 유사한 문서 검색
def search_similar_documents_by_text(collection_name: str, text: str):
    url = f"{BASE_URL}/document/{collection_name}/similar/text"
    data = {
        "text": text,  # [필수] 입력 텍스트
        "limit": 10,  # [필수] 검색할 문서 수
        "offset": 0,  # [필수] 페이징에 사용할 offset 값
        "with_metadata": True,  # [선택] 응답에 메타데이터 반환 여부
        "with_vector": False  # [선택] 응답에 내부 벡터 반환 여부
    }
    response = requests.post(url, headers=HEADERS, data=json.dumps(data))
    return response.json()

In [ ]:
# ID로 필터를 포함한 유사 문서 검색
# 필터를 사용하는 경우, 대상 필드에 인덱스 설정을 반드시 해야합니다.
# 필터 검색을 사용하고 싶은 경우 담당자에게 문의 주세요.
def search_similar_documents_with_filters(collection_name: str, doc_id: int, filter: dict):
    url = f"{BASE_URL}/document/{collection_name}/similar/{doc_id}"
    data = {
        "limit": 10,
        "offset": 0,
        "with_metadata": True,
        "with_vector": False,
        "filter": filter
    }
    response = requests.post(url, headers=HEADERS, data=json.dumps(data))
    return response.json()

In [ ]:
# 사용 예시
collection_name = "example_collection"
documents = [
    {"id": 1, "text": "The quick brown fox jumps over the lazy dog",
     "metadata": {"author": "John Doe", "category": "animal", "language": "English"}},
    {"id": 2, "text": "A quick movement of the enemy will jeopardize six gunboats",
     "metadata": {"category": "military", "language": "English"}},
    {"id": 3, "text": "All questions asked by five watched experts amaze the judge",
     "metadata": {"author": "John Doe", "category": "law", "language": "English"}},
    {"id": 4, "text": "The five boxing wizards jump quickly", "metadata": {"category": "sport", "language": "English"}},
    {"id": 5, "text": "How razorback-jumping frogs can level six piqued gymnasts",
     "metadata": {"author": "John Doe", "category": "animal", "language": "English"}},
    {"id": 6, "text": "Cozy lummox gives smart squid who asks for job pen",
     "metadata": {"category": "employment", "language": "English"}},
    {"id": 7, "text": "Jinxed wizards pluck ivy from the big quilt",
     "metadata": {"author": "John Doe", "category": "fantasy", "language": "English"}},
    {"id": 8, "text": "The job requires extra pluck and zeal from every young wage earner",
     "metadata": {"category": "employment", "language": "English"}},
    {"id": 9, "text": "A wizard’s job is to vex chumps quickly in fog",
     "metadata": {"author": "John Doe", "category": "fantasy", "language": "English"}},
    {"id": 10, "text": "Watch 'Jeopardy!', Alex Trebek's fun TV quiz game",
     "metadata": {"category": "entertainment", "language": "English"}}
]

In [ ]:
# 컬렉션 생성
print(create_collection(collection_name))

# 문서 추가
for doc in documents:
    print(add_document(collection_name, doc['id'], doc['text'], doc['metadata']))

# ID로 유사 문서 검색
print(search_similar_documents_by_id(collection_name, 1))

# 입력 텍스트와 유사한 문서 검색
print(search_similar_documents_by_text(collection_name, 'The slow red fox jumped over the fast dog'))

# 필터 예시 (author가 "John Doe"인 사용자)
filter_example = {
    "must": [  # AND 논리연산자와 유사
        {
            "key": "author",
            "match": {
                "value": "John Doe"
            }
        }
    ],
    "should": [  # OR 논리연산자와 유사

    ],
    "must_not": [  # NOT 논리연산자와 유사

    ]
}

# ID로 필터를 포함한 유사 문서 검색
print(search_similar_documents_with_filters(collection_name, 2, filter_example))